In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import warnings
warnings.simplefilter('ignore')

1.Visualization and EDA

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv('/kaggle/input/500-richest-people-2021/500 richest people 2021.csv',';')

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df=df.drop(['Unnamed: 7', 'Unnamed: 8','Unnamed: 9','Unnamed: 10'], axis=1)

In [ ]:
df.isnull().sum()

In [ ]:
df=df.dropna(how='all')

In [ ]:
df["Total Net Worth"] = df["Total Net Worth"].str.replace("$", "")
df["Total Net Worth"] = df["Total Net Worth"].str.replace("B", "")
df["$ Last Change"] = df["$ Last Change"].str.replace("$", "")
df["$ Last Change"] = df["$ Last Change"].str.replace("B", "")
df["$ Last Change"] = df["$ Last Change"].str.replace("M", "")
df["$ YTD Change"] = df["$ YTD Change"].str.replace("$", "")
df["$ YTD Change"] = df["$ YTD Change"].str.replace("B", "")
df["$ YTD Change"] = df["$ YTD Change"].str.replace("M", "")
df["Total Net Worth"] = df["Total Net Worth"].str.replace("family", "21")
df["$ Last Change"] = df["$ Last Change"].str.replace("k", "")
df["$ YTD Change"] = df["$ YTD Change"].str.replace("k", "")
df["$ Last Change"] = df["$ Last Change"].str.replace("+", "")
df["$ YTD Change"] = df["$ YTD Change"].str.replace("+", "")
df["Country"] = df["Country"].str.replace(" ", "")

In [ ]:
df['Total Net Worth']=df['Total Net Worth'].astype(float)
df['$ Last Change']=df['$ Last Change'].astype(float)
df['$ YTD Change']=df['$ YTD Change'].astype(float)

In [ ]:
df

In [ ]:
df.groupby('Country')['Total Net Worth'].sum().sort_values(ascending=False).plot.bar(figsize=(10,10))

Top 20 'Total Net Worth' by Countries

In [ ]:
import matplotlib.pyplot as plt
import plotly_express as px
df1=pd.DataFrame(df.groupby('Country')['Total Net Worth'].sum().sort_values(ascending=False).head(20))
df1['Country'] = df1.index
df1['Country codes']=['USA','CHN','FRA','RUS','GER','IND','HKG','GBR','JPN','MEX','AUS','SWE','SGP','ITA','CAN','ESP','BRA','CHE','KOR','IRL']
fig = px.choropleth(df1,locations='Country codes',color='Total Net Worth',scope='world')
fig.show()

In [ ]:
df.groupby('Country')['Name'].count().sort_values(ascending=False).plot.bar(figsize=(10,10))

Top 20 'Total of Names' by Countries

In [ ]:
df2=pd.DataFrame(df.groupby('Country')['Name'].count().sort_values(ascending=False).head(20))
df2['Country'] = df2.index
df2['Country codes']=['USA','CHN','GER','RUS','HKG','GBR','IND','FRA','CAN','SWE','SGP','CHE','AUS','ITA','KOR','JPN','DNK','MEX','BRA','TWN']
fig = px.choropleth(df2,locations='Country codes',color='Name',scope='world')
fig.show()

By Industries analysis

In [ ]:
df.groupby('Industry')['Total Net Worth'].sum().sort_values(ascending=False).plot.pie(figsize=(5,5),autopct="%1.1f%%")

In [ ]:
df.groupby('Industry')['Name'].count().sort_values(ascending=False).plot.pie(figsize=(5,5),autopct="%1.1f%%")

Scatterring 'Total Net Worth'by Indstries and Countries

In [ ]:
plt.figure(figsize=(12, 12))
plt.legend(fontsize=10)
plt.tick_params(labelsize=10)
ax=sns.scatterplot(x=df['Country'],y=df['Total Net Worth'],hue=df['Industry'],size=df['Total Net Worth'],data=df,sizes=(1,200))
plt.xticks(rotation=90)
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

In the US, a lot of rich people are in Technology which is defferent from France and Spain.
India and China have not had 'super rich' yet compared to the US and France.

2.Clustering

In [ ]:
df3=df.drop('Name',axis=1)

In [ ]:
df3

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le = le.fit(df3['Country'])
df3['Country_code'] = le.transform(df3['Country'])

In [ ]:
le2=le.fit(df3['Industry'])
df3['Industry_code']=le2.transform(df3['Industry'])

In [ ]:
df3

In [ ]:
df4=df3.drop(['Country','Industry'],axis=1)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
sc = StandardScaler()
cl_sc=sc.fit_transform(df4)
kmeans = KMeans(n_clusters=5, random_state=0)
clusters = kmeans.fit(cl_sc)

In [ ]:
df3["clustering"] = clusters.labels_
df3.head()

In [ ]:
df3.groupby('clustering')['Total Net Worth'].mean()

In [ ]:
df3.groupby('clustering')['$ Last Change'].mean()

In [ ]:
df3.groupby('clustering')['$ YTD Change'].mean()

In [ ]:
df4=pd.DataFrame(df3.groupby(['clustering','Country'])['Country'].count())
df4=df4.rename(columns={'Country':'num'})
df4 = df4.reset_index()

In [ ]:
df4

In [ ]:
plt.figure(figsize=(8, 8))
ax=sns.scatterplot(x=df4['Country'],y=df4['num'],hue=df4['clustering'],size=df4['num'],sizes=(0,50))
plt.xticks(rotation=90)
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

3.PCA Analysis

In [ ]:
pre = cl_sc
pca = PCA(n_components=2) 
pca.fit(pre)
pca_pre = pca.transform(pre)
pca_data = pd.DataFrame(pca_pre)

In [ ]:
for i in df3["clustering"].unique():
    tmp = pca_data.loc[df3["clustering"] == i]
    plt.scatter(tmp[0], tmp[1], label=i)
plt.legend()